In [1]:
import sys; sys.path.append("../")

import pandas as pd

from clean.post_pprr import clean

In [2]:
df = clean()
df

,department_desc,last_name,first_name,employment_status,level_1_cert_date,last_pc_12_qualification_date,hire_year,hire_month,hire_day,agency,data_production_year
0,14th jdc da office,abshire,dustin,full-time,06/11/1998,10/27/2010,2013,06,01,Peace Officer Standards and Training Council,2020
1,17th da office,ourso,damian,full-time,05/13/2004,10/07/2010,2008,11,11,Peace Officer Standards and Training Council,2020
2,17th da office,besson,bruce,full-time,NaN,01/26/2008,1997,02,11,Peace Officer Standards and Training Council,2020
3,17th da office,bourg,billy,full-time,04/24/1991,10/07/2010,1990,10,12,Peace Officer Standards and Training Council,2020
4,19th da office,cadarette,steve,full-time,08/28/2003,04/28/2020,2016,04,11,Peace Officer Standards and Training Council,2020
...,...,...,...,...,...,...,...,...,...,...,...
38898,youngsville pd,fruge,hilary,reserve,11/18/2010,06/22/2020,2017,11,27,Peace Officer Standards and Training Council,2020
38899,youngsville pd,mouton,todd,reserve,NaN,06/15/2020,2017,07,01,Peace Officer Standards and Training Council,2020
38900,youngsville pd,hidalgo,jeremy,reserve,NaN,07/31/2020,2017,12,01,Peace Officer Standards and Training Council,2020
38901,zachary pd,viccellio,david,reserve,NaN,12/04/2019,2019,12,03,Peace Officer Standards and Training Council,2020


In [3]:
df[df.first_name == ""]

,department_desc,last_name,first_name,employment_status,level_1_cert_date,last_pc_12_qualification_date,hire_year,hire_month,hire_day,agency,data_production_year
